In [9]:
from utils import read_lines
from collections import defaultdict
from threading import Thread
from queue import SimpleQueue, Empty
import time

def parse_program(s):
    return [int(x) for x in s.split(',')]

def parse_code(num):
    if num == 99:
        return num, [0]
    else:
        digits = []
        while num > 0:
            digits.append(num % 10)
            num //=10
        digits += [0] * (5-len(digits))
        opcode = digits[0]
        param_modes = digits[2:]
        return opcode, param_modes

class Computer:
    def __init__(self, program, id, queues):
        self.porgram = program
        self.id = id
        self.queues = queues
        self.q: SimpleQueue = queues[id]
        self.initialized = False

    def start(self):
        t = Thread(target=self.run)
        t.start()
        return t

    def run(self):
        # print(f'computer {self.id} started')
        i = 0
        outputs = []
        extend_memory = defaultdict(int)
        relative_base = 0
        inputs = None
        input_pos = 0
        program = self.porgram
        def read_memory(addr):
            if addr < len(program):
                return program[addr]
            else:
                return extend_memory[addr]
            
        def write_memory(idx, value, mode):
            addr = program[idx]
            if mode == 2:
                addr = relative_base + addr
            if addr < len(program):
                program[addr] = value
            else:
                extend_memory[addr] = value
        
        def get_oprand(idx, mode):
            if mode == 1:
                return program[idx]
            if mode == 2:
                addr = relative_base + program[idx]
            else:
                addr = program[idx]
            return read_memory(addr)

        while i < len(program):
            opcode, param_modes = parse_code(program[i])
            if opcode in (1, 2, 5, 6, 7, 8):
                oprand1 = get_oprand(i+1, param_modes[0])
                oprand2 = get_oprand(i+2, param_modes[1])
        
                if opcode == 1:
                    write_memory(i+3, oprand1 + oprand2, param_modes[2])
                    i+=4
                elif opcode == 2:
                    write_memory(i+3, oprand1 * oprand2, param_modes[2])
                    i+=4
                elif opcode == 5:
                    if oprand1 != 0:
                        i = oprand2
                    else:
                        i += 3
                elif opcode == 6:
                    if oprand1 == 0:
                        i = oprand2
                    else:
                        i += 3
                elif opcode == 7:
                    write_memory(i+3, 1 if oprand1 < oprand2 else 0, param_modes[2])
                    i+=4
                elif opcode == 8:
                    write_memory(i+3, 1 if oprand1 == oprand2 else 0, param_modes[2])
                    i+=4
            elif opcode ==3 :
                if not self.initialized:
                    input = self.id
                    self.initialized = True
                else:
                    if not inputs or input_pos == 2:
                        try:
                            inputs = self.q.get(True, 0.01)
                        except Empty:
                            inputs = None
                        else:
                            input_pos = 0
                    if inputs == 'stop':
                        # print(f'computer {self.id} stopped at stop message')
                        return
                    if not inputs:
                        input = -1
                    else:
                        input = inputs[input_pos]
                        input_pos += 1

                write_memory(i+1, input, param_modes[0])
                i += 2
            elif opcode == 4:
                oprand1 = get_oprand(i+1, param_modes[0])
                i += 2
                outputs.append(oprand1)
                if len(outputs) == 3:
                    print(f'computer {self.id} output {outputs}')
                    if outputs[0] == 255:
                        print(f'computer {self.id} got message to 255: {outputs}')
                        for q in self.queues.values():
                            q.put('stop')
                    else:
                        self.queues[outputs[0]].put(outputs[1:])
                        outputs = []
            elif opcode == 9:
                oprand1 = get_oprand(i+1, param_modes[0])
                relative_base += oprand1
                i+=2
            elif program[i] == 99:
                # print(f'computer {self.id} stopped at program 99')
                return outputs
            else:
                raise ValueError(f'illegal code {i} {program[i]}')

    
def part1(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    queues = {i: SimpleQueue() for i in range(50)}
    threads = []
    for i in range(50):
        computer = Computer(program.copy(), i, queues)
        t = computer.start()
        threads.append(t)
    for t in threads:
        t.join()




In [8]:
part1('inputs/day23.txt')

computer 0 output [7, 178138, 28774]computer 3 output [2, 141938, 36]
computer 4 output [10, 91199, 101]

computer 0 output [27, 96497, 28774]
computer 0 output [27, 289491, 28774]
computer 0 output [14, 19777, 28774]
computer 0 output [14, 39554, 28774]
computer 0 output [14, 59331, 28774]
computer 14 output [11, 98554, 23823234068824]
computer 5 output [16, 101281, -14]
computer 6 output [1, 2677, 1201]
computer 6 output [32, 134522, 1201]
computer 17 output [16, 303843, 3137]
computer 12 output [39, 266276, 7]
computer 9 output [32, 201783, -48]
computer 15 output [10, 364796, 2]
computer 15 output [13, 30922, 2]
computer 15 output [24, 142726, 2]
computer 15 output [24, 428178, 2]
computer 19 output [38, 90289, 353]
computer 8 output [18, 146702, -15]
computer 20 output [42, 136978, 256]
computer 22 output [18, 73351, 1283]
computer 21 output [13, 15461, 19]
computer 23 output [43, 194762, 10]
computer 23 output [24, 356815, 10]
computer 25 output [2, 70969, 1487]
computer 26 outpu

In [33]:
class Computer2:
    def __init__(self, program, id, queues, nat):
        self.porgram = program
        self.id = id
        self.queues = queues
        self.q: SimpleQueue = queues[id]
        self.initialized = False
        self.nat = nat

    def start(self):
        t = Thread(target=self.run)
        t.start()
        return t

    def run(self):
        # print(f'computer {self.id} started')
        i = 0
        outputs = []
        extend_memory = defaultdict(int)
        relative_base = 0
        inputs = None
        input_pos = 0
        program = self.porgram
        def read_memory(addr):
            if addr < len(program):
                return program[addr]
            else:
                return extend_memory[addr]
            
        def write_memory(idx, value, mode):
            addr = program[idx]
            if mode == 2:
                addr = relative_base + addr
            if addr < len(program):
                program[addr] = value
            else:
                extend_memory[addr] = value
        
        def get_oprand(idx, mode):
            if mode == 1:
                return program[idx]
            if mode == 2:
                addr = relative_base + program[idx]
            else:
                addr = program[idx]
            return read_memory(addr)

        while i < len(program):
            opcode, param_modes = parse_code(program[i])
            if opcode in (1, 2, 5, 6, 7, 8):
                oprand1 = get_oprand(i+1, param_modes[0])
                oprand2 = get_oprand(i+2, param_modes[1])
        
                if opcode == 1:
                    write_memory(i+3, oprand1 + oprand2, param_modes[2])
                    i+=4
                elif opcode == 2:
                    write_memory(i+3, oprand1 * oprand2, param_modes[2])
                    i+=4
                elif opcode == 5:
                    if oprand1 != 0:
                        i = oprand2
                    else:
                        i += 3
                elif opcode == 6:
                    if oprand1 == 0:
                        i = oprand2
                    else:
                        i += 3
                elif opcode == 7:
                    write_memory(i+3, 1 if oprand1 < oprand2 else 0, param_modes[2])
                    i+=4
                elif opcode == 8:
                    write_memory(i+3, 1 if oprand1 == oprand2 else 0, param_modes[2])
                    i+=4
            elif opcode ==3 :
                if not self.initialized:
                    input = self.id
                    self.initialized = True
                else:
                    if not inputs or input_pos == 2:
                        try:
                            inputs = self.q.get(True, 0.01)
                        except Empty:
                            inputs = None
                        else:
                            input_pos = 0
                            # print(f'{self.id} received {inputs}')
                    if inputs == 'stop':
                        # print(f'computer {self.id} stopped at stop message')
                        return
                    if not inputs:
                        input = -1
                    else:
                        input = inputs[input_pos]
                        input_pos += 1

                write_memory(i+1, input, param_modes[0])
                i += 2
            elif opcode == 4:
                oprand1 = get_oprand(i+1, param_modes[0])
                i += 2
                outputs.append(oprand1)
                if len(outputs) == 3:
                    # print(f'computer {self.id} output {outputs}')
                    if outputs[0] == 255:
                        # print(f'computer {self.id} got message to 255: {outputs}')
                        self.nat.set_packet(outputs[1:])
                        # for q in self.queues.values():
                        #     q.put('stop')

                    else:
                        self.queues[outputs[0]].put(outputs[1:])
                    outputs = []
            elif opcode == 9:
                oprand1 = get_oprand(i+1, param_modes[0])
                relative_base += oprand1
                i+=2
            elif program[i] == 99:
                # print(f'computer {self.id} stopped at program 99')
                return outputs
            else:
                raise ValueError(f'illegal code {i} {program[i]}')


class Nat:
    def __init__(self, queues):
        self.queues = queues
        self.cur_packet = None
        self.last_sent_packet = None

    def set_packet(self, packet):
        self.cur_packet = packet
        
    def start(self):
        t = Thread(target=self.run)
        t.start()
        return t

    def run(self):
        while True:
            if self.cur_packet and all(q.empty() for q in self.queues.values()):
                if self.last_sent_packet and self.last_sent_packet[1] == self.cur_packet[1]:
                    print(f'repeat nat message Y: {self.cur_packet[1]}')
                    for q in self.queues.values():
                        q.put('stop')
                    return
                self.queues[0].put(self.cur_packet[:])
                self.last_sent_packet = self.cur_packet[:]
                
            time.sleep(0.1)

def part2(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    queues = {i: SimpleQueue() for i in range(50)}
    nat = Nat(queues)
    t = nat.start()
    threads = [t]
    for i in range(50):
        computer = Computer2(program.copy(), i, queues, nat)
        t = computer.start()
        threads.append(t)
    for t in threads:
        t.join()

In [34]:
part2('inputs/day23.txt')

repeat nat message Y: 19216
